# 6교시 3.트랜스포머의 활용

### 위치인코딩과 정규화
#### positional encoding
- 사람이 개가 물다 != 사람이 개를 물다
- 단어의 어순이 바뀌면 의미가 완전히 달라진다.
- 내적을 하여 가중치만 나오기 때문에 순서가 무시된다. 언어의 중요 요소가 희석될 수 있기 때문에 토큰의 순서를 함께 포함시켜 학습.
- 순서 구하는 과정?
- : 단어의 위치를 나타내는 새로운 행렬을 임베딩 행렬과 같은 크기로 만들고, 이 행렬을 임베딩 행렬에 더하여(concat 아니라 수치적으로 plus) 단어의 위치 정보를 포함시킴.

- sin, cos 곡선을 이용해서 hidden_dim의 idx를 기준으로 짝수-sin/홀수-cos일때 나누어서 계산. 단, 짝수일때 sin이라도 곡선의 모양이 모두 달라서 encoding값이 다름. (예시 - 차원이 높아질수록 주기가 길어지도록 함)
-> 표현 다양성과 고유 위치 벡터 생성이 가능. 상대적 위치를 수학적으로 잘 표현할 수 있다.

- tahn의 경우, pos=100 == pos=1000, 긴 문장에서는 위치정보를 구분하지 못하는 문제. 대부분의 값이 중앙에 몰림.

####  정규화(norm)
- nn에서 입력값이 한쪽으로 치우쳐져 있으면 학습이 잘 되지 않음.
- 평균이 0, 분산이 1인 표준편차로
- 감마/베타는 학습으로 바꿔줌
- 감마: 폭, 베타: 좌우로 움직임
- **모든 데이터가 똑같이 정규화되지 않도록 학습가능한 파라미터로 특별하게 정규화를 진행함.** -> 고정되어있는 값이 아님
- 배치정규화:
배치 전체(훈련 하나 동안의 전체 샘플)를 하나의 집단으로보고, 그 안에서 정규화할 dim에 대해 평균/분산 계산. -> 동일한 위치의 속성을 정규화하여, 트랜스포머의 각 토큰이 다른 의미를 가질 때 효과적이지 않음. (각 샘플에서 같은 위치에 있는 애들을 정규화함 -> bn이 같은 위치에 있다는 이유로두 토큰을 강제 정규화함 -> 정규화로 독립적이어야 하는 각 토큰의 의미 정보가 희석될 수 있다.).
- 배치정규화 예시 ) ecg에서는 보통 channel을 축으로 하여 정규화 진행. (8, 12, 25, 250)에서 한 채널 내 8(배치)x20x250(length)값을 합쳐서 하나의 분포로 보고 평균과 분산을 계산.

-  이를 해결하고자 ** layer-wise normalization (special in Transformer)**: 한 토큰 벡터(예를 들어 한 문장에 토큰이 10개, (10, hidden_dim)이고 각 행이 한 토큰의 벡터이다)의 차원들 전체에서 평균과 분산을 계산
- (8, 12, 20, 250) ecg 데이터가 있을 때, 한 배치, 한 채널에서 20개의 패치 각각에 대해 250차원 벡터 하나씩 존재하고, 이 250차원 벡터가 정규화 대상.

#### 피드포원드 신경망
- 오토인코더: 중간에 노드 수를 줄이고 출력과 입력 노두 수를 똑같이 했을 때, 패턴을 파악 . 다시 복귀할 때 특징을 추론하여 차원을 줄이고 늘리는 과정에서 특징을 뽑아내게끔 하는 원리.
decoder -> output: 압축된 데이터를 원래 크기로 복원하면서 데이터 부족 & 특징을 잡아내는 오토인코더에 의한 이미지 생성이 가능했음.
- 이 오토인코더의 원리를 응용
- 512 -> ff (2046 node) -> 512
- 각 토큰의 임베딩을 독립적으로 변환함으로써 개별 단어의 특징을 더 잘 학습하게 됨.
- 내적으로 가중치 업데이트 + 임베딩 안에 위치 인코딩 + 노멀라이제이션 평준화 -> 피드포워드 신경망 통과 => 트랜스포머 알고리즘 안에서 모두 진행됨.

#### add (residual connection)
- multi-head attention 4x512 결과 (가중치가 포함) 에 처음 입력값을 더함.
- 왜? 가중치 Biased되거나 특이한 값이 있을 때 기울이 소실 문제를 완화시킴.


### maksed self-attention
- 디코더의 인풋으로 들어가는 값은 예측을 하기 전까지의 문장만 들어가야 하므로, 마스크를 씌워주어야 한다. (훈련 중 미래 토큰을 보지 못하게 막기 위해)
- 입력값: 디코더 자신의 입력 (지금까지 생성한 토큰) - 입력문장을 토큰화하고 임베딩해서 shifted right 해서 masking하고 다음 입력값을 예측함.
- 결과에서 지금 예측한거 다음부터는 다 잘라내고 지금 예측한것까지를 다시 써서 masking된 부분을 예측.
- 예) how about coffee? 커피 한잔 어때?
 => how about coffee? <sos> 커피 -- 까지 예측했다고 하면, 커피 뒷 부분은 잘라내고, 이것을 다시 입력으로 써서 그 다음 예측에는 '한잔'이라는 것이 나와야하도록 모델을 훈련해야하므로, '한잔'부분부터는 가린다.


아래 코드에는 'masked'라는 부분이 포함되어있음

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, LayerNormalization, Dropout, Add, Input, Lambda
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 깃허브에 준비된 데이터를 가져옵니다.
!git clone https://github.com/taehojo/data.git

# CSV 파일 로드
dataframe = pd.read_csv('./data/sentiment_data.csv')

# 데이터와 라벨 추출
sentences = dataframe['sentence'].tolist()
labels = dataframe['label'].tolist()

# 임베딩 벡터 크기와 최대 문장 길이 설정
embedding_dim = 128
max_len = 10 #나머지는 Padding

# 토크나이저 초기화 및 텍스트를 시퀀스로 변환
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sentences) #토큰화
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)
word_index = tokenizer.word_index #몇개? 임베딩 시 word_idx+1

# 패딩을 사용하여 시퀀스 길이를 동일하게 맞춤 #padding
data = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len, padding='post')

# 데이터셋을 훈련 세트와 검증 세트로 분리
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# 포지셔널 인코딩 함수
def get_positional_encoding(max_len, d_model):
    pos_enc = np.zeros((max_len, d_model))
    for pos in range(max_len):
        for i in range(0, d_model, 2):
            pos_enc[pos, i] = np.sin(pos / (10000 ** (2 * i / d_model)))
            if i + 1 < d_model: #i+1 홀수
                pos_enc[pos, i + 1] = np.cos(pos / (10000 ** (2 * (i + 1) / d_model)))
    return pos_enc

# 포지셔널 인코딩 생성
positional_encoding = get_positional_encoding(max_len, embedding_dim)

# 멀티헤드 어텐션 레이어
class MultiHeadSelfAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads, key_dim, masked=False):
        super(MultiHeadSelfAttentionLayer, self).__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)
        self.norm = LayerNormalization()
        self.masked = masked  # 새롭게 추가된 부분
#################################################################
    def call(self, x):
        if self.masked:
            # 마스크드 어텐션을 적용할 경우
            batch_size = tf.shape(x)[0]  # 입력 x의 배치 크기
            seq_len = tf.shape(x)[1]     # 입력 x의 시퀀스 길이

            # 마스크 행렬 생성
            mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)  #tf.linalg.band_part: 대각선 이전 값들을 1로 채우고 나머지는 0으로
            mask = tf.reshape(mask, (1, 1, seq_len, seq_len))
            # 마스크 행렬의 형태를 만들어 주는 부분 -->    # 만약 [1, 1, 4, 4] 라면
                                                        # [[1, 0, 0, 0],
                                                        #  [1, 1, 0, 0],
                                                        #  [1, 1, 1, 0],
                                                        #  [1, 1, 1, 1]] 이런 형태로 변환됨

            mask = tf.tile(mask, [batch_size, 1, 1, 1])  # 배치 크기만큼 마스크를 반복하여 확장

            # 마스크 행렬을 -무한대로 변경
            mask = mask * -1e9

            # 마스크를 사용한 멀티헤드 어텐션
            attn_output = self.mha(query=x, value=x, key=x, attention_mask=mask)
        else:
            # 마스크 없이 멀티헤드 어텐션을 적용할 경우
            attn_output = self.mha(query=x, value=x, key=x)

        attn_output = self.norm(attn_output + x) # 잔차 연결 적용 후 레이어 정규화

        return attn_output

# 모델 설정
inputs = Input(shape=(max_len,))

# 1. 임베딩 레이어: 텍스트 데이터를 임베딩 벡터로 변환합니다.
embedding_layer = Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim, input_length=max_len)
embedded_sequences = embedding_layer(inputs)

# 2. 포지셔널 인코딩 추가
embedded_sequences_with_positional_encoding = embedded_sequences + positional_encoding

# 3. 멀티헤드 어텐션 레이어 추가
attention_layer = MultiHeadSelfAttentionLayer(num_heads=8, key_dim=embedding_dim)
attention_output = attention_layer(embedded_sequences_with_positional_encoding)

# 4. 잔차 연결
attention_output_with_residual = Add()([embedded_sequences_with_positional_encoding, attention_output])

# 5. 마스크드 멀티헤드 어텐션 레이어 추가
masked_attention_layer = MultiHeadSelfAttentionLayer(num_heads=8, key_dim=embedding_dim, masked=True)  # masked=True 적용
masked_attention_output = masked_attention_layer(attention_output_with_residual)

# 6. 잔차 연결
masked_attention_output_with_residual = Add()([attention_output_with_residual, masked_attention_output])

# 7. GlobalAveragePooling1D 레이어 추가
pooled_output = GlobalAveragePooling1D()(masked_attention_output_with_residual)

# 8. 피드 포워드 네트워크
dense_layer = Dense(128, activation='relu')(pooled_output)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

# 모델 생성
model = Model(inputs=inputs, outputs=output_layer)

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train, np.array(y_train), epochs=10, batch_size=16, validation_data=(X_val, np.array(y_val)))

# 샘플 데이터 예측
sample_texts = ["I absolutely love this!", "I can't stand this product"]
sample_sequences = tokenizer.texts_to_sequences(sample_texts)
sample_data = tf.keras.preprocessing.sequence.pad_sequences(sample_sequences, maxlen=max_len, padding='post')
predictions = model.predict(sample_data)

for i, text in enumerate(sample_texts):
    print(f"Text: {text}")
    print(f"Prediction: {'Positive' if predictions[i] > 0.5 else 'Negative'}")


Cloning into 'data'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 36 (delta 9), reused 26 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 483.12 KiB | 6.44 MiB/s, done.
Resolving deltas: 100% (9/9), done.
[[11, 36, 9, 2], [25, 29], [109, 120, 14], [54, 12, 30, 78], [11, 97, 98, 9, 94], [102, 5], [24, 14], [6, 52], [103, 110], [111, 53], [92, 44], [9, 2, 4, 140], [11, 13, 6, 66, 10, 9], [21, 32, 5], [11, 13, 121], [112, 17, 95], [20], [11, 13, 141], [29, 33], [31, 30, 16, 51], [25, 36, 91], [113, 5], [142, 16, 53], [24, 93], [79, 27, 1, 78], [99, 1, 112], [29, 15, 5], [36, 1, 22], [143, 122], [97, 17, 94], [54, 17], [100, 114, 123], [21, 32, 28], [6, 144], [96, 5, 2], [58, 33], [26, 49], [6, 96, 5], [145, 5], [6, 146, 147], [20, 104], [25, 1, 112], [21, 148, 5], [6, 23], [56, 2], [36, 91], [54, 44], [6, 67], [24, 59], [29, 12], [20, 2], [24, 67], [6, 149], [25, 52]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.5150 - loss: 1.1658 - val_accuracy: 0.9950 - val_loss: 0.6479
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.8446 - loss: 0.3898 - val_accuracy: 1.0000 - val_loss: 0.0019
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9907 - loss: 0.0375 - val_accuracy: 1.0000 - val_loss: 0.0040
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9948 - loss: 0.0440 - val_accuracy: 1.0000 - val_loss: 0.0016
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9963 - loss: 0.0293 - val_accuracy: 1.0000 - val_loss: 0.0013
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9954 - loss: 0.0157 - val_accuracy: 1.0000 - val_loss: 9.9472e-05
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 1.0000 - val_loss: 1.4298e-05
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9997 - loss: 0.0038 - val_ac

<참고 도표>
데이터 특성과 모델 복잡성에 따라 마스크드 어텐션의 사용 여부를 결정해 최적의 모델을 선택하는 것이 중요합니다. 특히 마스크드 어텐션은 문장 순서가 중요한 경우에 유리합니다. 주어진 샘플의 경우, 아래와 같이 샘플 수를 줄이면 마스크드 어텐션 유무의 차이가 더욱 두드러지게 나타납니다. 이러한 요소가 주어진 데이터의 분석 성능을 향상시킬 수 있습니다.

<img src="https://github.com/taehojo/fastcampus_ai/blob/master/data/img/validation_accuracy_comparison_smooth.png?raw=1" alt="Validation Accuracy Comparison" width="600"/>
